# Trénovanie hlbokých sietí - optimalizácia a rady

Na minulom cvičení sme implementovali jednoduchú konvolučnú sieť a pri riešení vašich zadaní sa stretnete s ďalšími často používanými topológiami a architektúrami. Definovanie neurónovej siete je však iba prvý (a najjednoduchší) krok vývoja modelov, ktoré neskôr použijete v reálnych aplikáciách. Na to, aby ste vaše modely vedeli aj využiť, potrebujete ich vhodne natrénovať, tento proces je však skomplikovaný niekoľkými javmi. Cieľom dnešného cvičenia je oboznámiť vás s týmito problémami a základnými možnosťami ich vyriešenia.

Konkrétne sa pozrieme na hyperparametre (hlbokých) neurónových sietí, na vhodný výber ich hodnôt a na spôsoby urýchľovania trénovania hlbokých neurónových sietí, čo skoro vždy budete potrebovať, keďže výpočtové kapacity pre trénovanie sietí sú limitované.

## 1. Heuristika návrhu hlbokých sietí

| úloha                   | počet vstupov   | počet výstupov  | aktivačná funkcia vo výstupnej vrstve |
|-------------------------|-----------------|-----------------|---------------------------------------|
| binárna klasifikácia    | počet príznakov | 2               | sigmoid                               |
| multiclass klasifikácia | počet príznakov | C (počet tried) | softmax                               |
| predikcia               | počet príznakov | zvyčajne 1      | ReLU/linear                           |

Pri návrhu hlbokých sietí oplatí použiť veľa vrstiev iba v prípade konvolučných sietí, v plne prepojenej časti siete je odporúčané použiť max. 3 skryté vrstvy (ďalšie pridané skryté vrstvy nezvyšujú veľmi presnosť).

## 2. Trénovacie, testovacie a validačné množiny

Úspech, resp. neúspech trénovaných modelov závisí v prvom rade nielen od ich topológie a hyperparametrov, ale od použitých dát a ich rozdelenia. Najdôležitejším predpokladom pre úspešné nasadenie neurónových sietí je to, aby **príklady v trénovacích dátach zodpovedali očakávaným reálnym vstupným príkladom** pri používaní modelu na reálnu predikciu. Nesúlad môže spôsobiť problém napríklad pri trénovaní s obrázkami s vysokým rozlíšením a pri používaní na predikciu z obrázkov s nízkym rozlíšením a podobne.

Dáta v kontexte trénovania a testovania neurónových sietí zvyčajne rozdeľujeme na tri časti: **trénovacia** (*training*), **validačná** (*development* alebo *validation*) a **testovacia** (*test*) množina. Trénovacie dáta sa používajú na prispôsobovanie parametrov siete tak, aby predikcie siete boli čo najpresnejšie (fáza trénovania). Validačná množina slúži na vyhodnotenie presnosti siete ešte vo fáze trénovania, najmä na podporu rozhodnutia, či má ešte zmysel ďalej trénovať danú sieť. Testovacia množina napokon slúži na vyhodnotenie modelu z pohľadu používania - naším cieľom je nasimulovať čo najvhodnejšie prípady použitia.

V literatúre sa veľmi často stretnete s prípadom, kde sa použijú iba dve množiny, ktoré sú označené ako *trénovacia* a *testovacia* množina. Toto pomenúvanie je síce zaužívané, treba si ale uvedomiť, že *testovacia* množina v tomto kontexte skoro vždy hrá rolu *validačnej* množiny.

Základné odporúčanie na rozdelenie dostupných dát pre trénovanie neurónových sietí je *70/30%*, resp. *60/20/20%* v prípade rozdelenia do validačnej aj testovacej množiny. Toto rozdelenie slúži ako dobrá heuristika pre menšie datasety (desaťtisíce príkladov), avšak ak máte k dispozícii veľké dáta na trénovanie, priradenie *20-30%* príkladov do validačnej alebo testovacej množiny je často zbytočné.

V takýchto prípadoch treba brať do úvahy prvotný zámer použitia týchto množín - teda vyhodnotenie presnosti modelu, resp. porovnávanie dvoch modelov, a simulácia prípadov použitia. Práve preto ak máte státisíce alebo milióny údajov k dispozícii, veľmi často môžete priradiť až *90-98%* údajov do trénovacej množiny, a následne rozdeliť zvyšnú časť medzi validačnou a testovacou množinou (zvyčajne rovnomerne, alebo menej do testovacej).

### 2.1 Vyhodnotenie neurónovej siete

Spôsob vyhodnotenia natrénovaného modelu samozrejme závisí od konkrétneho príkladu použitia. Vo väčšine prípadov však vašou úlohou je dosiahnuť čo najvyššiu presnosť modelu a čo najmenšiu chybu. V iných prípadoch, kde cieľom je napríklad nahradiť existujúci proces, postačuje ak vaše riešenie prekonáva existujúce prístupy v istých metrikách.

Pri predikčných úlohách prvotnou metrikou je hodnota chyby, ktorú chcete znížiť. Pri klasifikácii správanie siete viete znázorniť pomocou konfúznej matice alebo kontingenčnej tabuľky. Konfúzna matica je tabuľková reprezentácia, kde v riadkoch máme očakávané triedy a v stĺpcoch vypočítané (predikované). V bunkách tabuľky sú uložené počty príkladov klasifikované v danej kombinácii očakávanej a predikovanej triedy. Ideálny klasifikátor bude mať všetky hodnoty po hlavnej diagonále (ďalšie informácie nájdete na [wikipédii](https://en.wikipedia.org/wiki/Confusion_matrix)).

Z konfúznej matice potom vieme vypočítať ďalšie metriky, ako správnosť (*accuracy*), návratnosť (*recall*) a presnosť (*precision*).

Správnosť popisuje samotný klasifikátor a vypočíta sa nasledovne:

$ACC = \frac{TP + TN}{P + N}$

kde $TP + TN$ je suma správne klasifikovaných príkladov (na hlavnej diagonále) a $P + N$ je počet všetkých príkladov.

Návratnosť (senzitivita) a presnosť popisujú klasifikátor pre danú triedu, vypočítajú sa nasledovne:

$REC = \frac{TP}{P}$

$PREC = \frac{TP}{TP + FP}$

kde $TP$ je počet správne klasifikovaných príkladov z danej triedy, $P$ je počet príkadov z danej triedy v testovacej množine a $FP$ je počet príkladov z testovacej množiny nesprávne klasifikovaných do tejto triedy.

Často sa vypočíta aj skóre F1, ktorá je harmonický priemer návratnosti a presnosti:

$F1 = 2 \cdot \frac{REC \cdot PREC}{REC + PREC}$

## 3. *Bias* a *variance*

Vývoj hlbokých modelov je iteratívny proces, pri ktorom navrhnete istú architektúru, vyhodnotíte ju, a na základe záveru vyhodnotenia prispôsobíte ho požiadavkám. V tomto smere vám pomôžu dva ukazovatele, a to *bias* a *variance* siete. *Bias* pritom vyjadruje schopnosť siete naučiť sa robiť presné predikcie, kým *variance* vyjadruje jej schopnosť generalizovať, t.j. zovšeobecniť predikciu na základe trénovacích dát. Tieto dva ukazovatele získate z číselných metrík - trénovacia (*training set error* - TSE) a validačná (*development set error* - DSE) chyba. Tieto chyby môžu byť vyjadrené ako priemerná chyba pri regresii, alebo percentuálna presnosť pri klasifikácii. Miera trénovacej chyby nám povie niečo o *biase*, kým porovnávanie trénovacej a validačnej chyby nám určí *variance*. Vysoký *bias* je dôsledkom podtrénovania, a vysoký *variance* zas pretrénovania.

Je dôležité povedať, že pri vývoji neurónových sietí by ste si mali určiť cieľovú presnosť, a rovnako uvažovať nad ľudskou presnosťou pre danú úlohu, resp. nad presnosťou náhodného klasifikátora. Pri vývoji sa najčastejšie stretnete s nasledovnými štyrmi prípadmi:

1. $TSE < DSE$, $TSE$ blízko alebo nižšia ako ľudská úroveň - váš model je pretrénovaný, nedokáže dobre generalizovať svoje znalosti na predikovanie; musíte skôr ukončiť trénovanie
2. $TSE \approx DSE$; nie sú blízko ľudskej úrovni - vysoký *bias*, váš model sa nedokáže natrénovať na trénovacích dátach; potrebujete viac údajov alebo vhodnejšie vybrať údaje, resp. predspracovať ich
3. $TSE>>, DSE>>$ - vysoký *bias* aj *variance*; váš model je pretrénovaný na niektoré časti údajov
4. $TSE<<, DSE<<$ - nízky *bias* aj *variance*; ideálny prípad.

Všeobecne vysoký *bias* riešite s robustnejšou topológiou, dlhším trénovaním, použitím iných optimalizačných algoritmov, alebo inou architektúrou siete. Vysoký *variance* sa rieši získaním väčšieho datasetu, regularizáciou údajov, a použitím inej architektúry.

## 4. Regularizácia

Cieľom regularizácie je predísť pretrénovaniu siete, t.j. chceme zabrániť tomu, aby ľubovoľný vstup, resp. ktorákoľvek váha ovplyvňovala výsledok vo vysokej, nadpriemernej miere. Pri použití regularizácie môžeme použiť aj väčšie siete bez rizika pretrénovania; čím väčšia je daná sieť, tým je menšia pravdepodobnosť, že regularizácia nám zvýši *bias*.

### 4.1. $L_{1}$ regularizácia

$L_{1}$ regularizácia pripočíta sumu absolútnych hodnôt váh do cieľovej funkcie, a tým zmenší koeficienty menej výrazných príznakov blízko 0. To má za následok, že niektoré príznaky sú ignorované, a sieť sa naučí riedšiu (*sparse*) reprezentáciu.

\begin{equation}
J(W, b) = \frac{1}{m} \sum_{i=1}^{m} L(\hat{y}^{2} - y^{2}) + \frac{\lambda}{2m} \sum_{l=1}^{L} \left \|w^{[l]}\right \|,
\end{equation}

kde $L$ je chybová funkcia, $\left \|w^{[l]}\right \|$ je suma absolútnych hodnôt všetkých váh a $\lambda$ je hyperparameter regularizácie.

### 4.2. $L_{2}$ regularizácia

$L_{2}$ regularizácia zabezpečuje, že hodnoty váh ostanú v istom intervale a tak sa sieť nebude nadmerne spoliehať na ani jednu z nich. Implementujeme ju rozšírením výpočty chyby siete:

\begin{equation}
J(W, b) = \frac{1}{m} \sum_{i=1}^{m} L(\hat{y}^{2} - y^{2}) + \frac{\lambda}{2m} \sum_{l=1}^{L} \left \|w^{[l]}\right \|_{F}^{2},
\end{equation}

kde $L$ je chybová funkcia, $\left \|w^{[l]}\right \|_{F}^{2}$ je tzv. Frobeniusova norma váh a $\lambda$ je hyperparameter regularizácie. Frobeniusova norma matice sa vypočíta nasledovne:

\begin{equation}
\left \|w^{[l]}\right \|_{F}^{2} = \sum_{i=1}^{n^{[l-1]}} \sum_{j=1}^{n^{[l]}}(w_{ij}^{[l]})^{2}
\end{equation}

pre *l*-tú vrstvu siete. Spôsob aktualizácia hodnoty váhy sa potom vykoná podľa vzorca:

\begin{equation}
\Delta w^{[l]} = \frac{\delta J}{\delta w^{[l]}} + \frac{\lambda}{m}w^{[l]}
\end{equation}

pre *l*-tú vrstvu siete.

Nevýhodou $L_{2}$ regularizácie je, že samotná $\lambda$ je ďalší hyperparameter, ktorého hodnotu vieme vhodne nastaviť len spôsobom pokus-omyl. Čím je $\lambda$ väčšia, tým viac sa budú hodnoty váh pohybovať okolo 0, tým pádom niektoré neuróny akokeby boli "vypnuté", vôbec ich sieť nebude používať - prakticky pracujeme s menšou sieťou. Ďalším dôsledkom použitia regularizácie je to, že suma vstupov neurónu bude takisto okolo 0, tým pádom sa použije iba lineárna časť aktivačných funkcií ako napr. sigmoidálna, a tým pádom nevieme pridať nelinearity do rozhodovania siete, teda zabránime pretrénovaniu.

$L_{2}$ regularizáciu v *PyTorchi* vieme použiť nastavením parametra `weight_decay` pre optimalizátor pri definícii trénovania (viď [dokumentácia napr. pre Adam](https://pytorch.org/docs/stable/generated/torch.optim.Adam.html#torch.optim.Adam)).

### 4.3. Dropout

Dropout je alternatívny spôsob regularizácie, kde pre niektoré vrstvy (alebo každú vrstvu) v sieti určíme pravdepodobnosť toho, že náhodne vypneme niektoré neuróny - to znamená, že odstránime všetky vstupné a výstupné synapsie daného neurónu pre konkrétny výpočet, pracujeme teda s menšou sieťou. Efekt tejto metódy je podobný ako v prípade $L_{2}$ regularizácie - sieť sa naučí nespoliehať sa na žiadny vstup príliš, keďže nemá garantované, že daný vstup bude vždy k dispozícii. Čím je viac pravdepodobné, že sa sieť v danej vrstve pretrénuje, tým vyššia by mala byť pravdepodobnosť *dropoutu*. Pravdepodobnosť 0.0 znamená, že *dropout* nepoužijeme.

*Dropout* sa najčastejšie použije v počítačovom videní, ak nemáme k dispozícii veľa trénovacích dát. Nevýhoda *dropoutu* je to, že chyba siete $J$ nie je jasne definovaná, a práve preto jej hodnota nemusí klesať po každej iterácii trénovania. *Dropout* sa zvyčajne nepoužije vo vstupnej vrstve, ak áno, tak s hodnotou okolo 0.0.

V *PyTorchi* *dropout* použijeme pridaním špeciálnej `Dropout` vrstvy ([dokumentácia](https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html)).

### 4.4. Augmentácia dát

Spôsobom riešenia malého množstva trénovacích údajov je augmentácia datasetu, teda rozšírenie datasetu algoritmickými spôsobmi. V prípade obrázkov je to najčastejšie prevrátenie obrazu (horizontálne alebo vertikálne, podľa typu dát), náhodné ostrihanie obrazu (sústrediť sa na stred obrazu), pridanie skreslení, rotácia obrazu, atď. Treba si dávať pozor, aby tieto úpravy mali zmysel v kontexte datasetu a netreba zabudnúť, že augmentovaný dataset nikdy nebude dosahovať kvalitu väčšieho datasetu s viacerými príkladmi.

### 4.5. Skoré ukončenie trénovania

Ak chceme predísť pretrénovaniu siete, stopercentná metóda je ukončenie trénovania vo vhodnom momente. Preto pri každej iterácii trénovania zmeriame trénovaciu a validačnú chybu, a trénovanie ukončíme, ak trénovacia chyba ešte klesá, avšak validačná chyba začne rásť. Nevýhodou tejto metódy je to, že nevieme rozdeliť proces optimalizácie chyby a zabránenie pretrénovaniu.

## 5. Minibatch trénovanie

Ako sme už písali vyššie, vývoj hlbokých modelov je iteratívny proces, a práve preto chceme modely natrénovať a vyhodnotiť čo najrýchlejšie. Na druhej strane ale hlboké učenie funguje pri veľkom množstve údajov. Keby sme váhy aktualizovali vždy nad celým datasetom pomocou *backpropagationu*, váhy by sme aktualizovali len pomaly a nezískali by sme rýchlo prehľad o efektívnosti modelu pre riešenie daného problému. Práve preto zavedieme *mini-batche*, a váhy budeme aktualizovať na základe vypočítaných chýb nad príkladmi z tejto dávky údajov.

Ak máme dataset *m* príkladov a nastavíme veľkosť *mini-batchov* na *m*, dostaneme základný *batch gradient descent* (vždy konverguje, avšak potrebuje veľa času medzi dvoma aktualizáciami). Ak veľkosť *mini-batchov* je 1, hovoríme o *stochastic gradient descent* (smeruje k lokálnemu minimu, nikdy nekonverguje, strácame možnosť paralelizovať). Veľký rozdiel medzi *batch* a *mini-batch* trénovaním je to, že pri *batch* trénovaní chyba bude nižšia po každej iterácii. Pri *mini-batch* trénovaní to nemusí platiť, keďže výber príkladov do *mini-batchu* je náhodný, chyba by ale stále mala mať klesajúcu tendenciu.

Pri malých datasetoch ($m < 2000$) môžeme použiť *batch* trénovanie, pri väčších datasetoch by sme mali využiť *mini-batche* (zvyčajne veľkosť 64, 128, 256, 512 - dávajte si pozor, aby sa jeden *mini-batch* zmestil do pamäte CPU/GPU). V *PyTorchi* *mini-batche* vieme zadefinovať pomocou parametra `batch_size` v `DataLoaderi` ([dokumentácia](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html)). Epocha potom reprezentuje prechádzanie celým datasetom.

## 6. Ladenie hyperparametrov

Ako vidíte, pri návrhu hlbokých sietí a pri ich trénovaní použijeme niekoľko hyperparametrov, ako napríklad: učiaci parameter, parametre optimalizátora, počet vrstiev, počet neurónov, *learning rate decay*, veľkosť *mini-batchu*. V poradí podľa dôležitosti (do akej miery ovplyvňujú presnosť modelu alebo rýchlosť trénovania):

1. učiaci parameter ($\alpha$)
2. veľkosť *mini-batchu*, počet neurónov, parametre optimalizátora
3. počet vrstiev, *learning rate decay*.

Otázka je, ako nájdeme vhodnú kombináciu hodnôt týchto parametrov pre efektívne trénovanie. Keďže neexistujú žiadne heuristiky a rady, ktoré by fungovali všeobecne, najjednoduchšia metóda je vyskúšať náhodné hodnoty parametrov, a porovnanie (krosvalidácia) presností modelov s danými parametrami. Týmto spôsobom jednoducho nájdeme, ktoré parametre ovplyvňujú úspech modelu najviac, a vieme sa sústrediť na ne. V prípade diskrétnych parametrov by sme si mali zvoliť vhodnú stupnicu možných hodnôt a z nej vybrať náhodné hodnoty - najčastejšie logaritmická. V Pythone viete náhodný výber z logaritmickej stupnice implementovať nasledovne (napr. pre interval $[10^{-4}, 10^0]$):

In [ ]:
r = -4 * np.random.ran()  # r in [-4, 0]
alpha = 10 ** r

Túto metódu môžete použiť pre učiaci parameter a parametre kĺzavého priemeru (optimalizátora - výber pre $1 - \beta$).

Ak nájdete oblasť v ktorom sa modely trénujú presnejšie, mali by ste sa sústrediť na ňu, a znova urobiť niekoľko iterácií náhodných výberov.

Ďalším dobrým krokom je hľadať vedecké články o podobných úlohách aj z iných domén, možno nájdete aplikovateľné riešenie. Vo fáze života modelu je treba niekoľkokrát preskúšať nastavenie hyperparametrov, možno dataset alebo počítačový systém sa zmenil.

## Použitá literatúra

* [Andrew Ng: Improving Deep Neural Networks - Hyperparameter Tuning](https://www.youtube.com/watch?v=1waHlpKiNyY&list=PLkDaE6sCZn6Hn0vK8co82zjQtt3T2Nkqc)